In [49]:
from nltk.tokenize import word_tokenize
import numpy as np
import json, os, torch
import pickle

In [113]:
maxSequenceLength = 100
unknown_threshold = 1

In [114]:
data_path = "/if24/tw8cb/PhD/software_eng/LanModeledProgramGeneartion-master/data"
word2idx = {}
idx2word = []
fre = {}
def add_word(word, word2idx, idx2word):
    if word not in word2idx:
        idx2word.append(word)
        word2idx[word] = len(idx2word) - 1

with open(os.path.join(data_path, 'train.data')) as f:
    for line in f:
        words = ['<SOS>'] + line.split() + ['<EOS>']
        if len(words) <= maxSequenceLength:
            for word in words:
                fre[word] = fre.get(word, 0) + 1
################visualization
counter=0
for key, value in sorted(fre.items(), key = lambda (k, v):(v, k)):
    if counter <= 5 or counter>len(fre) - 5:
        print(("{}: {}").format(key, value))
    counter += 1
#####################
print("fre size={}".format(len(fre)))
for (key, value) in fre.iteritems():
    if value > 0:
        add_word(key, word2idx, idx2word)
# add_word('<PAD>', word2idx, idx2word)
add_word('<UNKNOWN>', word2idx, idx2word)
print("dictionary size {}".format(len(word2idx)))

'"': 1
':': 1
'_': 1
'f': 1
's': 1
.new: 1
;: 49469
.: 51836
): 91099
(: 91654
fre size=18539
dictionary size 18540


In [118]:
pickle.dump(idx2word, open(os.path.join(data_path, "idx2word_100"), "wb"))

In [115]:
with open(os.path.join(data_path, "train.data"), 'r') as f:
    seq_len_fre = {}
    for line in f:
        words = ['<SOS>'] + line.split() + ['<EOS>']
        id_sen = list()
        token = 0
        for word in words:
            if word in word2idx:
                id_sen.append(word2idx[word])
            else:
                id_sen.append(word2idx['<UNKNOWN>'])
            token += 1
        seq_len_fre[token] = seq_len_fre.get(token, 1) + 1
print seq_len_fre

{6: 235, 7: 79, 8: 534, 9: 109, 10: 178, 11: 334, 12: 1113, 13: 503, 14: 817, 15: 668, 16: 963, 17: 670, 18: 888, 19: 588, 20: 677, 21: 577, 22: 527, 23: 503, 24: 584, 25: 485, 26: 445, 27: 400, 28: 461, 29: 379, 30: 349, 31: 308, 32: 345, 33: 273, 34: 837, 35: 236, 36: 259, 37: 266, 38: 269, 39: 385, 40: 207, 41: 197, 42: 186, 43: 385, 44: 266, 45: 182, 46: 942, 47: 159, 48: 199, 49: 210, 50: 192, 51: 307, 52: 166, 53: 152, 54: 206, 55: 166, 56: 158, 57: 192, 58: 135, 59: 158, 60: 125, 61: 118, 62: 127, 63: 119, 64: 116, 65: 103, 66: 124, 67: 104, 68: 127, 69: 106, 70: 106, 71: 117, 72: 92, 73: 103, 74: 71, 75: 99, 76: 113, 77: 88, 78: 91, 79: 78, 80: 100, 81: 79, 82: 88, 83: 74, 84: 80, 85: 63, 86: 62, 87: 63, 88: 69, 89: 88, 90: 53, 91: 50, 92: 67, 93: 46, 94: 51, 95: 59, 96: 50, 97: 52, 98: 56, 99: 67, 100: 56, 101: 51, 102: 55, 103: 48, 104: 61, 105: 52, 106: 54, 107: 39, 108: 40, 109: 44, 110: 42, 111: 53, 112: 37, 113: 43, 114: 45, 115: 39, 116: 33, 117: 50, 118: 30, 119: 38, 12

In [116]:
def make_sentences(split):
    with open(os.path.join(data_path, split), 'r') as f:
        ids = list()
        for line in f:
            words = ['<SOS>'] + line.split() + ['<EOS>']
            id_sen = list()
            token = 0
            for word in words:
                if word in word2idx:
                    id_sen.append(word2idx[word])
                else:
                    id_sen.append(word2idx['<UNKNOWN>'])
                token += 1
            if token <= maxSequenceLength:
                ids.append(id_sen)
#     for i in xrange(len(ids)):
#         sentence = ids[i]
#         if len(sentence) <= maxSequenceLength:
#             for j in xrange(maxSequenceLength - len(sentence)):
#                 sentence.append(word2idx['<PAD>'])
#         ids[i] = sentence
    print("number of {} datasets...{}".format(split, len(ids)))
    print("first sentence...{}".format(ids[0]))
    pickle.dump(ids, open(os.path.join(data_path, split+".sentences"), "wb"))
    return ids
ids = make_sentences('train.data')

number of train.data datasets...24414
first sentence...[7641, 11697, 3832, 8265, 8265, 8715, 2741, 2741, 13587, 3586, 6621, 17188, 744, 4913, 8715, 2741, 13587, 10180, 2741, 744, 2741, 8715, 13587, 744, 4581, 8715, 13587, 4093, 5752, 12244, 13330, 7074]


In [117]:
val_ids = make_sentences('val.data')
test_ids = make_sentences('test.data')

number of val.data datasets...2829
first sentence...[7641, 11697, 9341, 9341, 8715, 8571, 13965, 5042, 4333, 18539, 13587, 3586, 10123, 8715, 11972, 744, 1205, 744, 2188, 5101, 11972, 744, 12095, 744, 11666, 13587, 3586, 15836, 18539, 8715, 18539, 13587, 12244, 13330, 7746, 3586, 1243, 8715, 18539, 13587, 12244, 13330, 13330, 7074]
number of test.data datasets...2996
first sentence...[7641, 11697, 3832, 8743, 8743, 8715, 4628, 18539, 9622, 4581, 4581, 9622, 17265, 17265, 9622, 8265, 8265, 13587, 3586, 6621, 8743, 8715, 18539, 9622, 4581, 9622, 17265, 9622, 8265, 9622, 14437, 5752, 13587, 12244, 13330, 7074]


In [97]:
#don't need this
frequency = {}
for sentence in ids:
    for word in sentence:
        frequency[word] = frequency.get(word, 0) + 1
print(len(frequency))
print frequency[12694]
print frequency[12695]
total = 0
for item in frequency.values():
    total += item
print("sum is... {}".format(total))
weights = []
for key, value in frequency.iteritems():
    weights.append(float(total - value)/total)
weights[-2]=0.000000001
weights_new = []
total = 0
for item in weights:
    total += item
weights_new = []
for item in weights:
    weights_new.append(item / total * 12702)
pickle.dump(weights_new, open(os.path.join(data_path, 'weights'), 'wb'))
print weights_new

12701
3
2
sum is... 966200
[1.000235203683957, 1.0002341684570486, 1.0002310627763233, 1.0002341684570486, 1.0002341684570486, 1.0002341684570486, 1.000235203683957, 1.0002341684570486, 1.0002341684570486, 1.000235203683957, 1.0002341684570486, 1.000235203683957, 1.0002320980032315, 1.00023313323014, 1.000235203683957, 1.000230027549415, 1.000235203683957, 1.0002341684570486, 1.000235203683957, 1.0002320980032315, 1.0002320980032315, 1.0002320980032315, 1.000235203683957, 1.0002320980032315, 1.0002341684570486, 1.000235203683957, 1.0002320980032315, 1.000235203683957, 1.000235203683957, 1.000235203683957, 1.0002320980032315, 1.000230027549415, 1.0002320980032315, 1.0002341684570486, 1.0001824071116276, 1.0002289923225065, 1.000235203683957, 1.000235203683957, 1.000235203683957, 1.000235203683957, 1.0002341684570486, 1.000235203683957, 1.0002341684570486, 1.0002320980032315, 1.000227957095598, 1.0002341684570486, 1.000230027549415, 1.0002341684570486, 1.000235203683957, 1.00022795709559